## Предварительные данные

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

C:\Users\ASUS\anaconda3\envs\gpu2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\envs\gpu2\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\envs\gpu2\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3


In [3]:
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


In [4]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


In [5]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [6]:
merged_data = pd.merge(data, item_features, how='left')
merged_data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0,69,PRODUCE,Private,VEGETABLES - ALL OTHERS,CELERY,


In [7]:
merged_data.shape

(2396804, 18)

In [8]:
merged_data.loc[merged_data['quantity']==0]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
97,744,26985165432,1,5978648,0,0.0,31582,0.0,1119,1,0.0,0.0,1,,National,,,
128,1287,26985336468,1,5978648,0,0.0,304,0.0,1351,1,0.0,0.0,1,,National,,,
249,2305,26996870743,2,5978656,0,0.0,414,0.0,1300,1,-1.0,0.0,1,,National,,,
293,271,26997082949,2,5978656,0,0.0,329,0.0,1719,1,-2.0,0.0,1,,National,,,
694,315,27008952267,3,957951,0,0.0,327,0.0,1707,1,0.0,0.0,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395898,709,41653107125,663,1076056,0,0.0,400,0.0,1559,95,0.0,0.0,5,GROCERY,Private,COUPON/MISC ITEMS,MISC SALES TRANS,
2396003,338,41653143843,663,927090,0,0.0,369,0.0,1430,95,0.0,0.0,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB
2396151,430,41653163527,663,5978656,0,0.0,433,0.0,2039,95,-1.0,0.0,1,,National,,,
2396309,1326,41653184219,663,5978656,0,0.0,318,0.0,2319,95,-1.0,0.0,1,,National,,,


In [9]:
data_train = merged_data[merged_data['week_no'] < merged_data['week_no'].max() - test_size_weeks]
data_test = merged_data[merged_data['week_no'] >= merged_data['week_no'].max() - test_size_weeks]

In [10]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его существенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности?
- Какую метрику использовать?
- Сколько раз в неделю отпрпавляем рассылку?
- В какое время отправляем рассылку?
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

### **Теоретические предположения**

**3 товара с акцией** 

Если нужно, чтобы в топ-5 наиболее подходящих товаров попало три товара с акцией, то или организуется акция на какие-то из этих пяти, или берем первые топ-N наиболее подходящих, в которые попадают первые три акционных, на данный момент товара. Соответственно, получаем 3 товара с акцией и два первых неакционных товара из топ-5. Акционные товары имеют дополнительно весовую характеристику и соответственно, если меняется процент, то меняется и характеристика, что сказывается на дальнейшем выборе подходящих товаров.

**1 новый товар**

Товар должен быть рекомендован из товаров с наибоее близкими показателями (например, косинусное сходство) к его потребительской корзине (возможно новый сорт или новый вид/подвид какого-либо продукта с новыми качествами). Также можно сделать этот товар с акцией, тем самым мотивировать на покупку (переплетение с предыдущим пунктом - 3 товара с акцией)

**1 товар для роста среднего чека**

Средний чек - это соотношение суммы покупок к количеству заказов (средний доход с чека). Можно добавить новый товар со скидкой (переплетение с предыдущим пунктом - 1 новый товар), но который всё равно будет добавлять стоимости его корзине. Как было сказано в первом пункте - минимум на 3%.

### **Практические действия**

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

In [11]:
def prefilter_items(data):

    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))

    # Самые популярные товары
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() 
    popularity['share_unique_users'] = popularity['user_id'] / data['user_id'].nunique()
    
    # Уберем самые популярные товары
    top_popular = popularity[popularity['share_unique_users'] > 0.005].item_id.tolist()
    popul = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.0005].item_id.tolist()
    notpopul = popul[~popul['item_id'].isin(top_notpopular)]

    # Уберем не интересные для рекоммендаций категории (department). Оставим первые 10 категорий
    categories = data.groupby('department').count().reset_index().sort_values('item_id', ascending = False)[:10]
    categories = categories.department.tolist()
    res = notpopul[~notpopul['item_id'].isin(categories)]
    print(res.shape)

    # Уберем товары, которые не продавались за последние 12 месяцев
    res.drop(res[res.week_no > 52].index, inplace=True)

    # Уберем дешевые товары Покупки меньше 1$. 
    res.drop(res[res['price'] < 1].index, inplace=True)

    # Уберем дорогие товары. Покупки больше 100$.
    res.drop(res[res['price'] > 100].index, inplace=True)

    return res

   
def postfilter_items(user_id, recommednations):
    pass

In [12]:
data_train = prefilter_items(data_train)
data_train

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5576\3678830417.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


(206703, 19)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price
10,2375,26984851516,1,9487839,1,2.00,364,-0.79,1642,1,0.0,0.0,586,GROCERY,National,COOKIES/CONES,GRAHAM CRACKERS,11 OZ,2.00
13,1364,26984896261,1,920955,1,3.09,31742,0.00,1520,1,0.0,0.0,3664,MEAT,National,BEEF,GRND/PATTY - ROUND,,3.09
29,1172,26985025264,1,877180,1,2.29,396,0.00,946,1,0.0,0.0,2296,DRUG GM,National,SOAP - LIQUID & BAR,BODY WASH,,2.29
30,1172,26985025264,1,878664,1,2.00,396,-0.99,946,1,0.0,0.0,1297,DRUG GM,National,ORAL HYGIENE PRODUCTS,TOOTHPASTE,6 OZ,2.00
32,1172,26985025264,1,940796,1,3.59,396,0.00,946,1,0.0,0.0,2296,DRUG GM,National,HAIR CARE PRODUCTS,SHAMPOO,15 OZ,3.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185306,1103,32305240655,362,1016052,1,2.79,412,0.00,1023,52,0.0,0.0,69,NUTRITION,Private,CEREAL/BREAKFAST,CEREAL - COLD,10 OZ,2.79
1185324,1103,32305240655,362,7143071,1,2.99,412,0.00,1023,52,0.0,0.0,764,DRUG GM,National,ORAL HYGIENE PRODUCTS,TOOTHPASTE,,2.99
1185351,1122,32305244951,362,872098,1,3.99,412,-1.00,2048,52,0.0,0.0,1286,DRUG GM,National,HAIR CARE PRODUCTS,HAIR SETS AND GELS,2 OZ,3.99
1189498,1675,32308945672,362,323018,1,4.50,3235,-1.99,1309,52,0.0,0.0,2401,GROCERY,National,WATER - CARBONATED/FLVRD DRINK,SPRING WATER,405.6 OZ,4.50


In [13]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,25671,26691,27021,27745,27754,27812,27951,28165,28200,28219,...,13672286,13775195,13775715,13775931,13775932,13776505,13776765,13777140,13777572,13877013
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [15]:
userid_to_id[2]

1

In [16]:
csr_matrix(user_item_matrix).tocsr().shape[0]

2436

In [17]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)



  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: total: 1min 35s
Wall time: 45 s


In [18]:
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=sparse_user_item,   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

ValueError: user_items must contain 1 row for every user in userids

Все эти функции отправим затем в *utils.py*

**ДЗ**

1) Попытаться ответить на вопросы/выдвинуть гипотезы

*> e-mail мало кто читает, разве что рабочие. Зато все пользуются мессенджерами. +номер телефона при регистрации оставляют чаще, чем e-mail*<br>
*> слишком много уведомлений (письмо + смс + пуш + на чеке) тоже плохо, это отталкивает*<br>
*> реклама на чеке => увеличение расхода чековой бумаги => неэкологично*<br>
*> в смс не поместишь яркую картинку, что привлекает внимание; на чек тоже, чек никто не читает*<br>
*> пуш уведомления - только в приложении; приложениями пользуются только регулярные покупатели; чаще пользуются сайтом по мере необходимости*<br><br>

*> сортировать не только по вероятности; возможно какой-то товар люди не покупали, потому что дорогой, а со скидкой смогут себе позволить*<br>
*> рассылку отправлять 1 раз в неделю (чтобы ненапряжно); вечером, когда люди уже не заняты работой; перед выходными, когда у людей обычно есть время на закупки, в четверг или пятницу*<br>
*> возможно добавить проверку прошлых рекомендаций и исключать те, что были в прошлый раз*<br>
*> чтобы товары были разными, можно добавлять товары из разных категорий ИЛИ делать скидку только на ограниченное число товаров в рамках одной категории*<br><br>

**3 товара с акцией (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)**<br>
*последний вопрос не поняла*<br>
*с акцией лучше предлагать продукты изначально дорогие, т.к. изначально люди не могли их себе позволить, а со скидкой могут соблазниться*<br><br>

**1 новый товар (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)**<br>
*в дополнение к предложенному решению можно предлагать в принципе новый для магазина товар, как новинку; заодно и популяризуем/добавим инфы к новым товарам*<br><br>

**1 товар для роста среднего чека (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)**<br>
*можно предлагать товар, который юзер покупает чаще всего, но немножко дороже его обычного выбора*<br><br>




2) Доделать прошлые домашния задания


3) Прочитать статьи BM25/MatrixFactorization



4) Поэкспериментировать с ALS (grid-search)

In [ ]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

**Links**

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/